In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import statsmodels.api as sm
import scipy as sc
from scipy.stats import mode
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import Normalizer,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import gensim
import re
import gensim.downloader as model_api
from sklearn.model_selection import train_test_split

/Users/Coding/mambaforge/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
hvoe = pd.read_csv('data/HVOE.csv')
mo = pd.read_csv('data/Montreal Ouest.csv')
passionimo = pd.read_csv('data/Passionimo.csv')
monkland = pd.read_csv('data/Monkland.csv')
csl = pd.read_csv('data/CSL.csv')
ahc = pd.read_csv('data/Animal Health Clinic.csv')

df = pd.concat([hvoe, mo, passionimo, monkland, csl, ahc])
df

,Date,Name,Review,Rating
0,3 days ago,TheTabletguy,NaN,5 stars
1,a week ago,Jason Poirier Lavoie,NaN,5 stars
2,a week ago,Carlos Ceja cazares,Best care your family pet will get. The whole ...,5 stars
3,2 weeks ago,mara voltaire,"A very nice, clean place. Employées very polit...",5 stars
4,3 weeks ago,Samme Lem,(Translated by Google) This is the second time...,1 star
...,...,...,...,...
473,7 years ago,Liv L,I have been this clinic 4-5 times and with all...,2 stars
474,7 years ago,chloe rousseau,i had an amazing experience . the vet that hel...,5 stars
475,8 years ago,Stephanie Russell,I took my 8 year old cat to the Animal Health ...,5 stars
476,8 years ago,sandra joan,"i absolutely love this place,u can tell they r...",5 stars


In [3]:
df.Rating = df.Rating.str.replace('5 stars', '5')
df.Rating = df.Rating.str.replace('4 stars', '4')
df.Rating = df.Rating.str.replace('3 stars', '3')
df.Rating = df.Rating.str.replace('2 stars', '2')
df.Rating = df.Rating.str.replace('1 star', '1')
df.Rating = df.Rating.astype(int)
df

,Date,Name,Review,Rating
0,3 days ago,TheTabletguy,NaN,5
1,a week ago,Jason Poirier Lavoie,NaN,5
2,a week ago,Carlos Ceja cazares,Best care your family pet will get. The whole ...,5
3,2 weeks ago,mara voltaire,"A very nice, clean place. Employées very polit...",5
4,3 weeks ago,Samme Lem,(Translated by Google) This is the second time...,1
...,...,...,...,...
473,7 years ago,Liv L,I have been this clinic 4-5 times and with all...,2
474,7 years ago,chloe rousseau,i had an amazing experience . the vet that hel...,5
475,8 years ago,Stephanie Russell,I took my 8 year old cat to the Animal Health ...,5
476,8 years ago,sandra joan,"i absolutely love this place,u can tell they r...",5


In [4]:
df.Review = df.Review.astype(str)
df.Review = [x.lower() for x in df.Review]

In [5]:
df = df.dropna()
df = df.drop(['Date', 'Name'], axis = 1)

In [6]:
df_care = df[df['Review'].str.contains("care")]
df_care = df_care.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Care'})

df_cost = df[df['Review'].str.contains("cost")]
df_cost = df_cost.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Cost'})

df_service = df[df['Review'].str.contains("service")]
df_service = df_service.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Service'})

df_professional = df[df['Review'].str.contains("professional")]
df_professional = df_professional.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Professional'})

df_friendly = df[df['Review'].str.contains("friendly")]
df_friendly = df_friendly.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Friendly'})

df_tech = df[df['Review'].str.contains("technician")]
df_tech = df_tech.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Technician'})

df_vet = df[df['Review'].str.contains("vet")]
df_vet = df_vet.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Veterinarian'})

df_recep = df[df['Review'].str.contains("reception")]
df_recep = df_recep.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Receptionist'})

df_shap = pd.concat([df_care, df_cost, df_service, df_professional, df_friendly, df_tech, df_vet, df_recep], axis = 1)
df_shap['Average Rating'] = df_shap.apply(lambda x: 4.5, axis = 1)

df_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist,Average Rating
0,5.0,1.0,5.0,5.0,5.0,5.0,1,5.0,4.5
1,5.0,5.0,5.0,5.0,5.0,5.0,5,5.0,4.5
2,5.0,5.0,5.0,5.0,5.0,5.0,5,5.0,4.5
3,1.0,1.0,5.0,5.0,5.0,5.0,5,5.0,4.5
4,5.0,5.0,5.0,5.0,5.0,5.0,5,5.0,4.5
...,...,...,...,...,...,...,...,...,...
272,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,4.5
273,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,4.5
274,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,4.5
275,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,4.5


In [7]:
df_shap = df_shap.fillna(df_shap.mean())
df_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist,Average Rating
0,5.000000,1.000000,5.000000,5.00,5.000000,5.000000,1,5.0000,4.5
1,5.000000,5.000000,5.000000,5.00,5.000000,5.000000,5,5.0000,4.5
2,5.000000,5.000000,5.000000,5.00,5.000000,5.000000,5,5.0000,4.5
3,1.000000,1.000000,5.000000,5.00,5.000000,5.000000,5,5.0000,4.5
4,5.000000,5.000000,5.000000,5.00,5.000000,5.000000,5,5.0000,4.5
...,...,...,...,...,...,...,...,...,...
272,4.611111,2.473684,4.704918,4.75,4.889908,4.653846,4,3.9375,4.5
273,4.611111,2.473684,4.704918,4.75,4.889908,4.653846,5,3.9375,4.5
274,4.611111,2.473684,4.704918,4.75,4.889908,4.653846,5,3.9375,4.5
275,4.611111,2.473684,4.704918,4.75,4.889908,4.653846,5,3.9375,4.5


In [8]:
hvoe.Review = hvoe.Review.astype(str)
hvoe.Review = [x.lower() for x in hvoe.Review]

In [9]:
hvoe.Rating = hvoe.Rating.str.replace('5 stars', '5')
hvoe.Rating = hvoe.Rating.str.replace('4 stars', '4')
hvoe.Rating = hvoe.Rating.str.replace('3 stars', '3')
hvoe.Rating = hvoe.Rating.str.replace('2 stars', '2')
hvoe.Rating = hvoe.Rating.str.replace('1 star', '1')
hvoe.Rating = hvoe.Rating.astype(int)
hvoe

,Date,Name,Review,Rating
0,3 days ago,TheTabletguy,nan,5
1,a week ago,Jason Poirier Lavoie,nan,5
2,a week ago,Carlos Ceja cazares,best care your family pet will get. the whole ...,5
3,2 weeks ago,mara voltaire,"a very nice, clean place. employées very polit...",5
4,3 weeks ago,Samme Lem,(translated by google) this is the second time...,1
...,...,...,...,...
342,6 years ago,Danielle Poitras,(translated by google) i am a client of this c...,5
343,6 years ago,Vy Van,(translated by google) veterinary center laval...,1
344,6 years ago,h. muh,my sister took my bird there yesterday since h...,1
345,7 years ago,Jootchy,friendly staff and a veterinarian who know how...,5


In [10]:
hvoe = hvoe.dropna()
hvoe = hvoe.drop(['Date', 'Name'], axis = 1)

In [11]:
hvoe_care = hvoe[hvoe['Review'].str.contains("care")]
hvoe_care = hvoe_care.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Care'})

hvoe_cost = hvoe[hvoe['Review'].str.contains("cost")]
hvoe_cost = hvoe_cost.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Cost'})

hvoe_service = hvoe[hvoe['Review'].str.contains("service")]
hvoe_service = hvoe_service.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Service'})

hvoe_professional = hvoe[hvoe['Review'].str.contains("professional")]
hvoe_professional = hvoe_professional.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Professional'})

hvoe_friendly = hvoe[hvoe['Review'].str.contains("friendly")]
hvoe_friendly = hvoe_friendly.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Friendly'})

hvoe_tech = hvoe[hvoe['Review'].str.contains("technician")]
hvoe_tech = hvoe_tech.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Technician'})

hvoe_vet = hvoe[hvoe['Review'].str.contains("vet")]
hvoe_vet = hvoe_vet.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Veterinarian'})

hvoe_recep = hvoe[hvoe['Review'].str.contains("reception")]
hvoe_recep = hvoe_recep.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Receptionist'})

hvoe_shap = pd.concat([hvoe_care, hvoe_cost, hvoe_service, hvoe_professional, hvoe_friendly, hvoe_tech, hvoe_vet, hvoe_recep], axis = 1)

hvoe_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist
0,5.0,1.0,5.0,5.0,5.0,5.0,1,5.0
1,5.0,5.0,5.0,5.0,5.0,5.0,5,5.0
2,5.0,5.0,5.0,5.0,5.0,5.0,5,5.0
3,1.0,1.0,5.0,5.0,5.0,5.0,5,5.0
4,5.0,5.0,5.0,5.0,5.0,5.0,5,5.0
5,5.0,1.0,5.0,5.0,5.0,5.0,5,5.0
6,5.0,1.0,5.0,5.0,5.0,NaN,5,5.0
7,5.0,NaN,1.0,1.0,5.0,NaN,5,5.0
8,5.0,NaN,5.0,5.0,5.0,NaN,5,2.0
9,5.0,NaN,5.0,5.0,5.0,NaN,5,NaN


In [12]:
hvoe_shap = hvoe_shap.fillna(hvoe_shap.mean())
hvoe_shap['Average'] = hvoe_shap.mean(numeric_only=True, axis=1)
#hvoe_shap['Clinic'] = hvoe_shap.apply(lambda x: 'HVOE', axis = 1)
hvoe_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist,Average
0,5.000000,1.000000,5.00000,5.000000,5.0,5.0,1,5.000000,4.000000
1,5.000000,5.000000,5.00000,5.000000,5.0,5.0,5,5.000000,5.000000
2,5.000000,5.000000,5.00000,5.000000,5.0,5.0,5,5.000000,5.000000
3,1.000000,1.000000,5.00000,5.000000,5.0,5.0,5,5.000000,4.000000
4,5.000000,5.000000,5.00000,5.000000,5.0,5.0,5,5.000000,5.000000
5,5.000000,1.000000,5.00000,5.000000,5.0,5.0,5,5.000000,4.500000
6,5.000000,1.000000,5.00000,5.000000,5.0,5.0,5,5.000000,4.500000
7,5.000000,2.714286,1.00000,1.000000,5.0,5.0,5,5.000000,3.714286
8,5.000000,2.714286,5.00000,5.000000,5.0,5.0,5,2.000000,4.339286
9,5.000000,2.714286,5.00000,5.000000,5.0,5.0,5,4.666667,4.672619


In [13]:
hvoe_shap.to_csv('HVOE Shapley.csv', index=False, encoding='utf-8')

In [14]:
mo.Rating = mo.Rating.str.replace('5 stars', '5')
mo.Rating = mo.Rating.str.replace('4 stars', '4')
mo.Rating = mo.Rating.str.replace('3 stars', '3')
mo.Rating = mo.Rating.str.replace('2 stars', '2')
mo.Rating = mo.Rating.str.replace('1 star', '1')
mo.Rating = mo.Rating.astype(int)
mo

,Date,Name,Review,Rating
0,2 weeks ago,Christyn Greczkowski,NaN,5
1,a month ago,Christian Fritschi,NaN,5
2,a month ago,Carlos Ceja cazares,"Excellent service, all recommendations and tre...",5
3,a month ago,Cindy Ibberson,"Dr Bernier has the experience, expertise and k...",5
4,2 months ago,Richard Fuller,We had to bring our dog into the UK which was ...,5
...,...,...,...,...
63,4 years ago,Kent Conover,NaN,4
64,4 years ago,Annie Perreault,NaN,5
65,4 years ago,Matthew Ward,NaN,5
66,4 years ago,Serge Bouharevich,Dr. Bernier and his staff are responsible and ...,5


In [15]:
mo = mo.dropna()
mo = mo.drop(['Date', 'Name'], axis = 1)

In [16]:
mo_care = mo[mo['Review'].str.contains("care")]
mo_care = mo_care.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Care'})

mo_cost = mo[mo['Review'].str.contains("cost")]
mo_cost = mo_cost.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Cost'})

mo_service = mo[mo['Review'].str.contains("service")]
mo_service = mo_service.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Service'})

mo_professional = mo[mo['Review'].str.contains("professional")]
mo_professional = mo_professional.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Professional'})

mo_friendly = mo[mo['Review'].str.contains("friendly")]
mo_friendly = mo_friendly.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Friendly'})

mo_tech = mo[mo['Review'].str.contains("technician")]
mo_tech = mo_tech.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Technician'})

mo_vet = mo[mo['Review'].str.contains("vet")]
mo_vet = mo_vet.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Veterinarian'})

mo_recep = mo[mo['Review'].str.contains("reception")]
mo_recep = mo_recep.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Receptionist'})

mo_shap = pd.concat([mo_care, mo_cost, mo_service, mo_professional, mo_friendly, mo_tech, mo_vet, mo_recep], axis = 1)

mo_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist
0,5.0,5.0,5.0,5.0,NaN,NaN,5,1.0
1,5.0,NaN,5.0,5.0,NaN,NaN,1,NaN
2,5.0,NaN,2.0,5.0,NaN,NaN,5,NaN
3,NaN,NaN,5.0,5.0,NaN,NaN,5,NaN
4,NaN,NaN,5.0,5.0,NaN,NaN,5,NaN
5,NaN,NaN,5.0,5.0,NaN,NaN,2,NaN
6,NaN,NaN,5.0,NaN,NaN,NaN,5,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN


In [17]:
passionimo.Review = passionimo.Review.astype(str)
passionimo.Review = [x.lower() for x in passionimo.Review]

In [18]:
passionimo.Rating = passionimo.Rating.str.replace('5 stars', '5')
passionimo.Rating = passionimo.Rating.str.replace('4 stars', '4')
passionimo.Rating = passionimo.Rating.str.replace('3 stars', '3')
passionimo.Rating = passionimo.Rating.str.replace('2 stars', '2')
passionimo.Rating = passionimo.Rating.str.replace('1 star', '1')
passionimo.Rating = passionimo.Rating.astype(int)
passionimo

,Date,Name,Review,Rating
0,6 days ago,Maude R. V.,nan,5
1,a week ago,amanda faigal,i've been a few times now for my guineapigs an...,5
2,4 weeks ago,Sophie A. D.,(translated by google) i would like to thank t...,5
3,a month ago,Yvonne Gardner,great service. friendly helpful staff.,5
4,a month ago,nancy boudrault,(translated by google) very professional servi...,5
...,...,...,...,...
260,4 years ago,Suzie Brouillard,(translated by google) i am rather sad. i like...,3
261,4 years ago,Atef Sdiri,nan,1
262,4 years ago,Daniel Montpetit,(translated by google) great team of veterinar...,5
263,4 years ago,Michelle Franklin,excellent service. we've taken all our pets to...,5


In [19]:
passionimo = passionimo.dropna()
passionimo = passionimo.drop(['Date', 'Name'], axis = 1)

In [20]:
passionimo_care = passionimo[passionimo['Review'].str.contains("care")]
passionimo_care = passionimo_care.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Care'})

passionimo_cost = passionimo[passionimo['Review'].str.contains("cost")]
passionimo_cost = passionimo_cost.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Cost'})

passionimo_service = passionimo[passionimo['Review'].str.contains("service")]
passionimo_service = passionimo_service.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Service'})

passionimo_professional = passionimo[passionimo['Review'].str.contains("professional")]
passionimo_professional = passionimo_professional.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Professional'})

passionimo_friendly = passionimo[passionimo['Review'].str.contains("friendly")]
passionimo_friendly = passionimo_friendly.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Friendly'})

passionimo_tech = passionimo[passionimo['Review'].str.contains("technician")]
passionimo_tech = passionimo_tech.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Technician'})

passionimo_vet = passionimo[passionimo['Review'].str.contains("vet")]
passionimo_vet = passionimo_vet.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Veterinarian'})

passionimo_recep = passionimo[passionimo['Review'].str.contains("reception")]
passionimo_recep = passionimo_recep.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Receptionist'})

passionimo_shap = pd.concat([passionimo_care, passionimo_cost, passionimo_service, passionimo_professional, passionimo_friendly, passionimo_tech, passionimo_vet, passionimo_recep], axis = 1)

passionimo_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist
0,5.0,1.0,5,5.0,5.0,5.0,5.0,1.0
1,5.0,2.0,5,5.0,5.0,5.0,5.0,5.0
2,5.0,1.0,5,5.0,2.0,5.0,5.0,5.0
3,5.0,1.0,5,5.0,5.0,5.0,5.0,5.0
4,5.0,NaN,5,5.0,5.0,5.0,5.0,5.0
5,5.0,NaN,2,5.0,5.0,5.0,5.0,5.0
6,5.0,NaN,5,5.0,5.0,5.0,5.0,NaN
7,5.0,NaN,5,5.0,5.0,5.0,5.0,NaN
8,5.0,NaN,5,5.0,5.0,NaN,5.0,NaN
9,5.0,NaN,5,5.0,5.0,NaN,2.0,NaN


In [21]:
passionimo_shap = passionimo_shap.fillna(passionimo_shap.mean())
passionimo_shap['Average'] = passionimo_shap.mean(numeric_only=True, axis=1)
passionimo_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist,Average
0,5.000,1.00,5,5.0,5.0000,5.0,5.00,1.000000,4.000000
1,5.000,2.00,5,5.0,5.0000,5.0,5.00,5.000000,4.625000
2,5.000,1.00,5,5.0,2.0000,5.0,5.00,5.000000,4.125000
3,5.000,1.00,5,5.0,5.0000,5.0,5.00,5.000000,4.500000
4,5.000,1.25,5,5.0,5.0000,5.0,5.00,5.000000,4.531250
5,5.000,1.25,2,5.0,5.0000,5.0,5.00,5.000000,4.156250
6,5.000,1.25,5,5.0,5.0000,5.0,5.00,4.333333,4.447917
7,5.000,1.25,5,5.0,5.0000,5.0,5.00,4.333333,4.447917
8,5.000,1.25,5,5.0,5.0000,5.0,5.00,4.333333,4.447917
9,5.000,1.25,5,5.0,5.0000,5.0,2.00,4.333333,4.072917


In [22]:
monkland.Rating = monkland.Rating.str.replace('5 stars', '5')
monkland.Rating = monkland.Rating.str.replace('4 stars', '4')
monkland.Rating = monkland.Rating.str.replace('3 stars', '3')
monkland.Rating = monkland.Rating.str.replace('2 stars', '2')
monkland.Rating = monkland.Rating.str.replace('1 star', '1')
monkland.Rating = monkland.Rating.astype(int)
monkland

,Date,Name,Review,Rating
0,a day ago,tom Morris,"300 dollars to put my cat down,absolutely app...",1
1,5 days ago,julie pinard,Je vais à cette clinique depuis son ouverture ...,5
2,a week ago,Ben M,NaN,5
3,2 weeks ago,Bryn Williams-Jones,"Excellent customer service, quick appointments...",5
4,3 weeks ago,Hugo Tardy,(Translated by Google) My dog ​​has been a pat...,5
...,...,...,...,...
195,4 years ago,Graeme Bent,A little pricey but professional!,4
196,4 years ago,Cristian M,"Professional staff, a little bit expensive but...",4
197,5 years ago,Jeff Langsner,First thing you'll notice is that it's nice in...,5
198,5 years ago,Charles Tatossian,Great veterinarian. Easy to book an appointment.,5


In [23]:
monkland.Review = monkland.Review.astype(str)
monkland.Review = [x.lower() for x in monkland.Review]

monkland = monkland.dropna()
monkland = monkland.drop(['Date', 'Name'], axis = 1)

In [24]:
monkland_care = monkland[monkland['Review'].str.contains("care")]
monkland_care = monkland_care.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Care'})

monkland_cost = monkland[monkland['Review'].str.contains("cost")]
monkland_cost = monkland_cost.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Cost'})

monkland_service = monkland[monkland['Review'].str.contains("service")]
monkland_service = monkland_service.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Service'})

monkland_professional = monkland[monkland['Review'].str.contains("professional")]
monkland_professional = monkland_professional.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Professional'})

monkland_friendly = monkland[monkland['Review'].str.contains("friendly")]
monkland_friendly = monkland_friendly.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Friendly'})

monkland_tech = monkland[monkland['Review'].str.contains("technician")]
monkland_tech = monkland_tech.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Technician'})

monkland_vet = monkland[monkland['Review'].str.contains("vet")]
monkland_vet = monkland_vet.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Veterinarian'})

monkland_recep = monkland[monkland['Review'].str.contains("reception")]
monkland_recep = monkland_recep.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Receptionist'})

monkland_shap = pd.concat([monkland_care, monkland_cost, monkland_service, monkland_professional, monkland_friendly, monkland_tech, monkland_vet, monkland_recep], axis = 1)

monkland_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist
0,5.0,1.0,5.0,5.0,5.0,5.0,5,5.0
1,5.0,2.0,5.0,5.0,5.0,5.0,5,5.0
2,5.0,4.0,4.0,5.0,5.0,5.0,5,5.0
3,5.0,NaN,5.0,5.0,5.0,5.0,5,NaN
4,5.0,NaN,5.0,4.0,5.0,4.0,5,NaN
5,5.0,NaN,5.0,5.0,5.0,NaN,5,NaN
6,5.0,NaN,5.0,4.0,5.0,NaN,5,NaN
7,4.0,NaN,5.0,5.0,4.0,NaN,5,NaN
8,5.0,NaN,5.0,5.0,5.0,NaN,5,NaN
9,5.0,NaN,5.0,5.0,5.0,NaN,5,NaN


In [25]:
monkland_shap = monkland_shap.fillna(monkland_shap.mean())
monkland_shap['Average'] = monkland_shap.mean(numeric_only=True, axis=1)
monkland_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist,Average
0,5.00,1.000000,5.000000,5.000000,5.000000,5.0,5,5.0,4.500000
1,5.00,2.000000,5.000000,5.000000,5.000000,5.0,5,5.0,4.625000
2,5.00,4.000000,4.000000,5.000000,5.000000,5.0,5,5.0,4.750000
3,5.00,2.333333,5.000000,5.000000,5.000000,5.0,5,5.0,4.666667
4,5.00,2.333333,5.000000,4.000000,5.000000,4.0,5,5.0,4.416667
5,5.00,2.333333,5.000000,5.000000,5.000000,4.8,5,5.0,4.641667
6,5.00,2.333333,5.000000,4.000000,5.000000,4.8,5,5.0,4.516667
7,4.00,2.333333,5.000000,5.000000,4.000000,4.8,5,5.0,4.391667
8,5.00,2.333333,5.000000,5.000000,5.000000,4.8,5,5.0,4.641667
9,5.00,2.333333,5.000000,5.000000,5.000000,4.8,5,5.0,4.641667


In [26]:
csl

,Date,Name,Review,Rating
0,a month ago,Catherine Daigneault,(Translated by Google) The vet was very friend...,4 stars
1,a month ago,gilles paquette,NaN,4 stars
2,a month ago,Sherika Duncan,Great hospital for my little doggies. Friendly...,5 stars
3,2 months ago,susan montgrain,They were very efficient and called a taxi for...,5 stars
4,3 months ago,Maya D.,NaN,5 stars
...,...,...,...,...
98,4 years ago,User Name,Toke an appointment still waited over an hour ...,1 star
99,5 years ago,Spaghetti Sandwich,This clinic is the best. They are honest and c...,5 stars
100,6 years ago,Florent Gatore,NaN,5 stars
101,7 years ago,john ben,"Best Vets in the City ,thank you for all you do .",5 stars


In [27]:
csl.Rating = csl.Rating.str.replace('5 stars', '5')
csl.Rating = csl.Rating.str.replace('4 stars', '4')
csl.Rating = csl.Rating.str.replace('3 stars', '3')
csl.Rating = csl.Rating.str.replace('2 stars', '2')
csl.Rating = csl.Rating.str.replace('1 star', '1')
csl.Rating = csl.Rating.astype(int)

csl.Review = csl.Review.astype(str)
csl.Review = [x.lower() for x in csl.Review]

csl = csl.dropna()
csl = csl.drop(['Date', 'Name'], axis = 1)

csl_care = csl[csl['Review'].str.contains("care")]
csl_care = csl_care.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Care'})

csl_cost = csl[csl['Review'].str.contains("cost")]
csl_cost = csl_cost.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Cost'})

csl_service = csl[csl['Review'].str.contains("service")]
csl_service = csl_service.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Service'})

csl_professional = csl[csl['Review'].str.contains("professional")]
csl_professional = csl_professional.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Professional'})

csl_friendly = csl[csl['Review'].str.contains("friendly")]
csl_friendly = csl_friendly.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Friendly'})

csl_tech = csl[csl['Review'].str.contains("technician")]
csl_tech = csl_tech.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Technician'})

csl_vet = csl[csl['Review'].str.contains("vet")]
csl_vet = csl_vet.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Veterinarian'})

csl_recep = csl[csl['Review'].str.contains("reception")]
csl_recep = csl_recep.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Receptionist'})

csl_shap = pd.concat([csl_care, csl_cost, csl_service, csl_professional, csl_friendly, csl_tech, csl_vet, csl_recep], axis = 1)

csl_shap = csl_shap.fillna(csl_shap.mean())
csl_shap['Average'] = csl_shap.mean(numeric_only=True, axis=1)

csl_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist,Average
0,5.000000,NaN,5.0,1.000000,4.000000,5.0,4,4.000000,4.000000
1,5.000000,NaN,5.0,4.000000,5.000000,1.0,5,3.000000,4.000000
2,5.000000,NaN,5.0,5.000000,5.000000,3.0,1,1.000000,3.571429
3,5.000000,NaN,5.0,5.000000,5.000000,3.0,5,2.666667,4.380952
4,5.000000,NaN,5.0,5.000000,5.000000,3.0,4,2.666667,4.238095
5,5.000000,NaN,5.0,5.000000,3.000000,3.0,5,2.666667,4.095238
6,5.000000,NaN,5.0,4.166667,5.000000,3.0,5,2.666667,4.261905
7,1.000000,NaN,5.0,4.166667,4.000000,3.0,5,2.666667,3.547619
8,5.000000,NaN,5.0,4.166667,5.000000,3.0,5,2.666667,4.261905
9,4.555556,NaN,5.0,4.166667,5.000000,3.0,5,2.666667,4.198413


In [28]:
ahc.Rating = ahc.Rating.str.replace('5 stars', '5')
ahc.Rating = ahc.Rating.str.replace('4 stars', '4')
ahc.Rating = ahc.Rating.str.replace('3 stars', '3')
ahc.Rating = ahc.Rating.str.replace('2 stars', '2')
ahc.Rating = ahc.Rating.str.replace('1 star', '1')
ahc.Rating = ahc.Rating.astype(int)

ahc.Review = ahc.Review.astype(str)
ahc.Review = [x.lower() for x in ahc.Review]

ahc = ahc.dropna()
ahc = ahc.drop(['Date', 'Name'], axis = 1)

ahc_care = ahc[ahc['Review'].str.contains("care")]
ahc_care = ahc_care.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Care'})

ahc_cost = ahc[ahc['Review'].str.contains("cost")]
ahc_cost = ahc_cost.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Cost'})

ahc_service = ahc[ahc['Review'].str.contains("service")]
ahc_service = ahc_service.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Service'})

ahc_professional = ahc[ahc['Review'].str.contains("professional")]
ahc_professional = ahc_professional.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Professional'})

ahc_friendly = ahc[ahc['Review'].str.contains("friendly")]
ahc_friendly = ahc_friendly.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Friendly'})

ahc_tech = ahc[ahc['Review'].str.contains("technician")]
ahc_tech = ahc_tech.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Technician'})

ahc_vet = ahc[ahc['Review'].str.contains("vet")]
ahc_vet = ahc_vet.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Veterinarian'})

ahc_recep = ahc[ahc['Review'].str.contains("reception")]
ahc_recep = ahc_recep.reset_index().drop(['index', 'Review'], axis = 1).rename(columns = {'Rating' : 'Receptionist'})

ahc_shap = pd.concat([ahc_care, ahc_cost, ahc_service, ahc_professional, ahc_friendly, ahc_tech, ahc_vet, ahc_recep], axis = 1)

ahc_shap = ahc_shap.fillna(ahc_shap.mean())
ahc_shap['Average'] = ahc_shap.mean(numeric_only=True, axis=1)

ahc_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist,Average
0,1.00,1.00,5.000000,5.000000,5.000000,5.0,5,5.0,4.000000
1,5.00,1.00,5.000000,5.000000,5.000000,1.0,5,5.0,4.000000
2,5.00,5.00,5.000000,5.000000,5.000000,5.0,1,4.0,4.375000
3,5.00,4.00,5.000000,5.000000,5.000000,5.0,5,5.0,4.875000
4,5.00,2.75,5.000000,5.000000,5.000000,5.0,5,2.0,4.343750
...,...,...,...,...,...,...,...,...,...
89,4.35,2.75,4.531915,4.592593,4.894737,4.2,4,3.4,4.089906
90,4.35,2.75,4.531915,4.592593,4.894737,4.2,5,3.4,4.214906
91,4.35,2.75,4.531915,4.592593,4.894737,4.2,5,3.4,4.214906
92,4.35,2.75,4.531915,4.592593,4.894737,4.2,5,3.4,4.214906


In [29]:
combined_shap = pd.concat([hvoe_shap, monkland_shap, ahc_shap, csl_shap, mo_shap, passionimo_shap])
combined_shap = combined_shap.fillna(monkland_shap.mean())
combined_shap['Average Rating'] = combined_shap.mean(numeric_only=True, axis=1)
combined_shap = combined_shap.drop('Average', axis = 1)
combined_shap

,Care,Cost,Service,Professional,Friendly,Technician,Veterinarian,Receptionist,Average Rating
0,5.000,1.00,5.0,5.0,5.0000,5.0,1.00,5.000000,4.000000
1,5.000,5.00,5.0,5.0,5.0000,5.0,5.00,5.000000,5.000000
2,5.000,5.00,5.0,5.0,5.0000,5.0,5.00,5.000000,5.000000
3,1.000,1.00,5.0,5.0,5.0000,5.0,5.00,5.000000,4.000000
4,5.000,5.00,5.0,5.0,5.0000,5.0,5.00,5.000000,5.000000
...,...,...,...,...,...,...,...,...,...
47,4.875,1.25,5.0,5.0,4.8125,5.0,5.00,4.333333,4.408854
48,4.875,1.25,5.0,5.0,4.8125,5.0,5.00,4.333333,4.408854
49,4.875,1.25,1.0,5.0,4.8125,5.0,5.00,4.333333,3.908854
50,4.875,1.25,5.0,5.0,4.8125,5.0,4.64,4.333333,4.363854


In [30]:
combined_shap.isnull().values.sum()

0

In [31]:
#combined_shap.to_csv('data/Combined Shapley.csv', index=False, encoding='utf-8')